## Predicting Movie Success

In [1]:
#Dependencies
from pathlib import Path
import pandas as pd

In [2]:
data = Path('movies_dataset_2.csv')
df = pd.read_csv(data)
df.head(3)

,imdb_id,title,release_year,runtime,genre,overview,director,actors,rated,imdb_votes,...,budget,revenue,star_1,star_2,star_3,genre_1,genre_2,genre_3,director_1,director_2
0,tt8936646,Extraction,2020,116 min,"Action, Crime, Thriller",A hardened gun-for-hire's latest mission becom...,Sam Hargrave,"Chris Hemsworth, Bryon Lerum, Ryder Lerum",R,"267,724",...,65000000,0,Chris Hemsworth,Bryon Lerum,Ryder Lerum,Action,Crime,Thriller,Sam Hargrave,NaN
1,tt2737304,Bird Box,2018,124 min,"Horror, Mystery, Sci-Fi",Five years after an ominous unseen presence dr...,Susanne Bier,"Sandra Bullock, Trevante Rhodes, John Malkovich",R,"392,201",...,19800000,0,Sandra Bullock,Trevante Rhodes,John Malkovich,Horror,Mystery,Sci-Fi,Susanne Bier,NaN
2,tt0328107,Man on Fire,2004,146 min,"Action, Crime, Drama",Jaded ex-CIA operative John Creasy reluctantly...,Tony Scott,"Denzel Washington, Christopher Walken, Dakota ...",R,"394,563",...,70000000,130300000,Denzel Washington,Christopher Walken,Dakota Fanning,Action,Crime,Drama,Tony Scott,NaN


In [3]:
df.columns

Index(['imdb_id', 'title', 'release_year', 'runtime', 'genre', 'overview',
       'director', 'actors', 'rated', 'imdb_votes', 'popularity', 'budget',
       'revenue', 'star_1', 'star_2', 'star_3', 'genre_1', 'genre_2',
       'genre_3', 'director_1', 'director_2'],
      dtype='object')

In [4]:
df.drop(['imdb_id', 'title', 'actors', 'director', 'overview', 'genre', 'revenue', 'imdb_votes','popularity','revenue'], axis=1, inplace=True)

In [5]:
df.head()

,release_year,runtime,rated,budget,star_1,star_2,star_3,genre_1,genre_2,genre_3,director_1,director_2
0,2020,116 min,R,65000000,Chris Hemsworth,Bryon Lerum,Ryder Lerum,Action,Crime,Thriller,Sam Hargrave,NaN
1,2018,124 min,R,19800000,Sandra Bullock,Trevante Rhodes,John Malkovich,Horror,Mystery,Sci-Fi,Susanne Bier,NaN
2,2004,146 min,R,70000000,Denzel Washington,Christopher Walken,Dakota Fanning,Action,Crime,Drama,Tony Scott,NaN
3,2016,120 min,PG-13,110000000,Shailene Woodley,Theo James,Jeff Daniels,Action,Adventure,Mystery,Robert Schwentke,NaN
4,1987,103 min,R,28000000,Eddie Murphy,Judge Reinhold,Jürgen Prochnow,Action,Comedy,Crime,Tony Scott,NaN


In [6]:
def runtime_cleaner (string):
    minutes = string.split(" ")[0]
    return int(minutes)

df['runtime'] = df['runtime'].map(runtime_cleaner)

In [7]:
df['rated'].unique()

array(['R', 'PG-13', 'TV-14', 'G', 'PG', 'Not Rated', 'Approved', 'TV-MA',
       'NC-17', 'Unrated', nan], dtype=object)

In [8]:
rated_list = df['rated'].unique().tolist()

for i, rating in enumerate(rated_list):
    df['rated'].replace(rating, int(i), inplace=True)


In [9]:
for i, rating in enumerate(rated_list):
    df['rated'].replace(rating, int(i), inplace=True)

df.head()    

,release_year,runtime,rated,budget,star_1,star_2,star_3,genre_1,genre_2,genre_3,director_1,director_2
0,2020,116,0.0,65000000,Chris Hemsworth,Bryon Lerum,Ryder Lerum,Action,Crime,Thriller,Sam Hargrave,NaN
1,2018,124,0.0,19800000,Sandra Bullock,Trevante Rhodes,John Malkovich,Horror,Mystery,Sci-Fi,Susanne Bier,NaN
2,2004,146,0.0,70000000,Denzel Washington,Christopher Walken,Dakota Fanning,Action,Crime,Drama,Tony Scott,NaN
3,2016,120,1.0,110000000,Shailene Woodley,Theo James,Jeff Daniels,Action,Adventure,Mystery,Robert Schwentke,NaN
4,1987,103,0.0,28000000,Eddie Murphy,Judge Reinhold,Jürgen Prochnow,Action,Comedy,Crime,Tony Scott,NaN


In [10]:
stars_list = [*df["star_1"].tolist(),*df["star_2"].tolist(),*df["star_3"].tolist()]

unique_stars_list = []

for star in stars_list:
    if star not in unique_stars_list:
        unique_stars_list.append(star)
    
print(len(unique_stars_list))

for i, star in enumerate(unique_stars_list):
    df['star_1'].replace(star, int(i), inplace=True)
    df['star_2'].replace(star, int(i), inplace=True)
    df['star_3'].replace(star, int(i), inplace=True)

df.head()  


1623


,release_year,runtime,rated,budget,star_1,star_2,star_3,genre_1,genre_2,genre_3,director_1,director_2
0,2020,116,0.0,65000000,0,474,1091,Action,Crime,Thriller,Sam Hargrave,NaN
1,2018,124,0.0,19800000,1,475,982,Horror,Mystery,Sci-Fi,Susanne Bier,NaN
2,2004,146,0.0,70000000,2,476,1092,Action,Crime,Drama,Tony Scott,NaN
3,2016,120,1.0,110000000,3,477,531,Action,Adventure,Mystery,Robert Schwentke,NaN
4,1987,103,0.0,28000000,4,478,1093,Action,Comedy,Crime,Tony Scott,NaN


In [15]:
genre_list = [*df["genre_1"].tolist(),*df["genre_2"].tolist(),*df["genre_3"]]

unique_genre_list = []

for genre in genre_list:
    if genre not in unique_genre_list:
        unique_genre_list.append(genre)
    
print(len(unique_genre_list))

for i, genre in enumerate(unique_genre_list):
    df['genre_1'].replace(genre, int(i), inplace=True)
    df['genre_2'].replace(genre, int(i), inplace=True)
    df['genre_3'].replace(genre, int(i), inplace=True)

df.head()    

51


,release_year,runtime,rated,budget,star_1,star_2,star_3,genre_1,genre_2,genre_3,director_1,director_2
0,2020,116,0.0,65000000,0,474,1091,0,12,32,0,527
1,2018,124,0.0,19800000,1,475,982,1,13,33,1,527
2,2004,146,0.0,70000000,2,476,1092,0,12,34,2,527
3,2016,120,1.0,110000000,3,477,531,0,14,35,3,527
4,1987,103,0.0,28000000,4,478,1093,0,15,36,2,527


In [16]:
director_list = [*df["director_1"].tolist(),*df["director_2"].tolist()]

unique_director_list = []

for director in director_list:
    if director not in unique_director_list:
        unique_director_list.append(director)
    
print(len(unique_director_list))

for i, director in enumerate(unique_director_list):
    df['director_1'].replace(director, int(i), inplace=True)
    df['director_2'].replace(director, int(i), inplace=True)

df.head()    

594


,release_year,runtime,rated,budget,star_1,star_2,star_3,genre_1,genre_2,genre_3,director_1,director_2
0,2020,116,0.0,65000000,0,474,1091,0,12,32,0,527
1,2018,124,0.0,19800000,1,475,982,1,13,33,1,527
2,2004,146,0.0,70000000,2,476,1092,0,12,34,2,527
3,2016,120,1.0,110000000,3,477,531,0,14,35,3,527
4,1987,103,0.0,28000000,4,478,1093,0,15,36,2,527


## Separate the Features(X) from the Target (y)


 ## Split our data into training and testing

 ## Create a Logistic Regression Model

 ## Fit (train) or model using the training data

 ## Score the model using the test data

 ## Make predictions

## Calculate the Accuracy Score